# Obtenção e carregamento dos dados

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# Importação do arquivo CSV
df = pd.read_csv('../Data/dataset_clustered.csv')

# Definição e tratamento dos dados de entrada.

## Informações sobre o dataset

In [ ]:
df.info()
df.head()

## Tratamento de valores ausentes

In [ ]:
#preenche com o texto "Abandono" os valores nulos da coluna 'categoria_motivo_cancelamento' se a coluna evasao for igual a 1, caso contrário deixa vazio
df['categoria_motivo_cancelamento'] = df['categoria_motivo_cancelamento'].fillna('Abandono').where(df['evasao'] == 1, 'NA')

df.info()

## Separação de variaveis categoricas e numericas

In [ ]:
# Defina as colunas numéricas e categóricas
num_cols = (df.select_dtypes(include=['int64', 'float64']).columns).where(df.select_dtypes(include=['int64', 'float64']).columns != 'evasao').dropna().tolist()
cat_cols = ['genero', 'ds_ensino_medio', 'ds_ingresso', 'cd_curso', 'situacao', 'categoria_motivo_cancelamento']

# Análise dos clusters

## Identificação e distribuição dos clusters	

In [ ]:
n_clusters = df['cluster'].nunique()

# Calcula o total de registros
total = len(df)
# Criando uma lista de cores para cada cluster
colors=[ '#FFC75F', '#FF9671', '#FF6F91', '#D65DB1', '#845EC2']
plt.figure(figsize=(10, 6))
df['cluster'].value_counts().plot(kind='pie', autopct=lambda p: '{:.0f}\n({:.1f}%)'.format((p/100)*total,p), startangle=90, shadow=False, legend=False, fontsize=12, colors=colors)
plt.title('Distribuição de alunos por cluster')

labels = []
for i in range(n_clusters):
    labels.append(f'Cluster {i}')

# Cria uma lista de patches para a legenda
patches = [mpatches.Patch(color=color, label=label) for color, label in zip(colors, labels)]

plt.legend(handles=patches, loc='upper right')
plt.axis('equal')
plt.ylabel('')
plt.show()


## Estudo das variáveis

### Média de cada variável por cluster

In [ ]:
# Calcula a média de cada variavel numérica por cluster
df.groupby('cluster')[num_cols].mean().round(2)

### Evasão por cluster

In [ ]:
# Cálculo da média de evasão para cada cluster
evasao_por_cluster = pd.DataFrame(df.groupby('cluster')['evasao'].mean().round(2))
evasao_por_cluster.head()

## Perfil dos clusters

### Características dos clusters

#### Componentes numéricos

##### Descrição dos componentes por cluster

In [ ]:
# Para as características numéricas
df_desc_num = pd.DataFrame(df.groupby('cluster')[num_cols].describe().T)
df_desc_num.head(16)

##### Distribuição dos componentes numéricos por cluster

In [ ]:
df_no_outliers = pd.DataFrame()
# Para cada coluna numérica
for col in num_cols:
    # Para cada cluster
    for i in range(n_clusters):
        cluster_data = df[df['cluster'] == i]
        
        Q1 = cluster_data[col].quantile(0.25)
        Q3 = cluster_data[col].quantile(0.75)
        IQR = Q3 - Q1

        # Somente mantém valores dentro de 1.5*IQR da mediana
        filtered_data = cluster_data[(cluster_data[col] >= Q1 - 1.5*IQR) & (cluster_data[col] <= Q3 + 1.5*IQR)]
        
        # Adicione os dados filtrados ao novo DataFrame
        df_no_outliers = pd.concat([df_no_outliers, filtered_data], axis=0)

for col in num_cols:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='cluster', y=col, data=df_no_outliers, palette=colors)
    plt.title(f'Distribuição de {col} por cluster (IQR)')
    plt.show()

#### Componentes categóricos

##### Total de componentes por cluster

In [ ]:
# Para as características categóricas
df_cat = pd.DataFrame()
for col in cat_cols:
    temp_df = df.groupby('cluster')[col].value_counts().unstack().fillna(0)
    temp_df.columns = [f'{col}_{c}' for c in temp_df.columns]
    df_cat = pd.concat([df_cat, temp_df], axis=1)
df_cat = df_cat.T

df_cat.head(16)

##### Descrição dos componentes categoricos por cluster

In [ ]:
# Para cada coluna categórica
for col in cat_cols:
    df_cat_col = df.groupby('cluster')[col].value_counts().unstack().fillna(0)
    
    df_cat_col.plot(kind='barh', stacked=True, figsize=(20, 6))
    plt.title(f'Distribuição de {col} por cluster')
    plt.legend(bbox_to_anchor=(-0.01, -0.05), loc='upper left', ncol=9) 
    plt.show()

#### Componente evasão

In [ ]:
# Calcule a contagem de evasores para cada cluster
evasao_counts = df[df['evasao'] == 1].groupby('cluster').size()

# Crie um gráfico de barras para a contagem de evasores
evasao_counts.plot(kind='bar', figsize=(10, 6))
plt.title('Distribuição dos alunos evasores por cluster')
plt.xlabel('Cluster')
plt.ylabel('Alunos evasores')
plt.show()
